In [2]:
#pip install cvzone

In [1]:
import cv2
import pickle
import cvzone

In [8]:
width,height=60,30

try:
    with open('carparkpos','rb') as f:
        position=pickle.load(f)
except:
        position=[]
def mouseclick(event,x,y,flags,params):
    if event==cv2.EVENT_LBUTTONDOWN:
        position.append((x,y))
    if event==cv2.EVENT_RBUTTONDOWN:
        for i,poss in enumerate(position):
            x1,y1=poss
            if x1<x<x1+width and y1<y<y1+height:
                position.pop(i)
    with open('carparkpos','wb') as f:
        pickle.dump(position,f)

        
while True:
    img=cv2.imread('carpark.png')
    img=cv2.resize(img,(700,500))

    for pos in position:
        cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),(255,0,255),2)

    cv2.imshow('img',img)
    cv2.setMouseCallback('img',mouseclick)

    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()

In [2]:
#we can find the parking space is free or not by looking for edges
#if there are very much edges are present then there is an car is parked.

In [34]:
cap=cv2.VideoCapture('carPark.mp4')

width,height=60,30
kernel=np.zeros((3,3),np.uint8)
with open('carparkpos','rb') as f:
    position=pickle.load(f)

def checkparkingspace(imageprocessed):
    spacecounter=0
    for pos in position:
        
        x,y=pos
        #cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),(255,0,255),2)
        cv2.imshow('Image',img)
        imgcrop=imageprocessed[y:y+height,x:x+width]
        #cv2.imshow(str(x+y),imgcrop)
        count=cv2.countNonZero(imgcrop)
        cvzone.putTextRect(img,str(count),(x,y+height+5),scale=1,thickness=2,offset=0,colorR=(0,255,0))
        if count<200:
            color=(255,0,0)
            thick=1
            spacecounter+=1
            cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),color,thick)
        else:
            color=(0,0,250)
            thick=1
            cv2.rectangle(img,pos,(pos[0]+width,pos[1]+height),color,thick)
        cvzone.putTextRect(img,f'Parking Available{str(spacecounter)} / {len(position)}',(150,50),scale=2,thickness=2,offset=0,colorR=(203,135,130))
while True:
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES,0)
    success,img=cap.read()
    img=cv2.resize(img,(700,500))
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(3,3),1)
    thresh=cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,16)
    medianblur=cv2.medianBlur(thresh,5)
    dilate=cv2.dilate(medianblur,kernel,iterations=1)
    #cv2.imshow('video',img)
    checkparkingspace(dilate)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>